In [127]:
import pandas as pd
from utils.data_preprocessor import DataPreprocessor

In [128]:
df = pd.read_csv('../data/preprocessed_data.csv')

In [129]:
df.shape

(3900, 16)

### Transformation of dataset before clustering:

1. **Label Encoding** for Ordinal Columns: Preserves the order of the categories.
2. **One-Hot Encoding** for Nominal Columns: Ensures no ordinal relationship is imposed.
3. **Normalization** of Numerical Columns: Standardizes the numerical data.

## Define ordered, categorical and numerical columns

In [130]:
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
numerical_cols = df.select_dtypes(include=['float64', 'int64']).columns.tolist()

# check if the length of categorical and numerical columns is equal to the total number of columns
len(categorical_cols) + len(numerical_cols) == df.shape[1]

True

In [131]:
# look for some ordered categorical columns
for col in categorical_cols:
    print('-' * 50)
    print(col)
    print()
    print(df[col].value_counts())

--------------------------------------------------
Gender

Male      2652
Female    1248
Name: Gender, dtype: int64
--------------------------------------------------
Category

Clothing       1737
Accessories    1240
Footwear        599
Outerwear       324
Name: Category, dtype: int64
--------------------------------------------------
Location

Montana           96
California        95
Idaho             93
Illinois          92
Alabama           89
Minnesota         88
Nebraska          87
New York          87
Nevada            87
Maryland          86
Delaware          86
Vermont           85
Louisiana         84
North Dakota      83
Missouri          81
West Virginia     81
New Mexico        81
Mississippi       80
Indiana           79
Georgia           79
Kentucky          79
Arkansas          79
North Carolina    78
Connecticut       78
Virginia          77
Ohio              77
Tennessee         77
Texas             77
Maine             77
South Carolina    76
Colorado          75
Ok

In [132]:
# declare 2 ordered categorical columns which need to be handled
ordered_categorical_cols = ['Size', 'Frequency of Purchases']

# remove ordered categorical columns from the list of categorical columns
categorical_cols = [col for col in categorical_cols if col not in ordered_categorical_cols]

In [133]:
cols = ordered_categorical_cols + categorical_cols + numerical_cols

In [134]:
# change the order of columns to have numerical first and categorical last
df = df[cols]

## Transform variables

### Transform ordered columns - Size

In [135]:
ordered_categorical_cols

['Size', 'Frequency of Purchases']

In [136]:
df['Size'].value_counts()

M     1755
L     1053
S      663
XL     429
Name: Size, dtype: int64

In [137]:
# Define the order of the sizes
size_order = ['S', 'M', 'L', 'XL']

# Create a dictionary to map each size to an integer
size_mapping = {size: index + 1 for index, size in enumerate(size_order)}

# Map the size column using the defined order
df['Size'] = df['Size'].map(size_mapping)

### Transform ordered columns - Frequency of Purchases

We can observe that there are 2 values which means the same: Quarterly and Every 3 Months, we combine them to 1 category

In [138]:
df['Frequency of Purchases'].value_counts()

Every 3 Months    584
Annually          572
Quarterly         563
Monthly           553
Bi-Weekly         547
Fortnightly       542
Weekly            539
Name: Frequency of Purchases, dtype: int64

In [139]:
frequency_mapping = {
    'Weekly': 6,
    'Fortnightly': 5,
    'Bi-Weekly': 4,
    'Monthly': 3,
    'Quarterly': 2,             # combine 'Quarterly' and 'Every 3 Months'       
    'Every 3 Months': 2,        # combine 'Quarterly' and 'Every 3 Months'
    'Annually': 1
}

df['Frequency of Purchases'] = df['Frequency of Purchases'].map(frequency_mapping)

### Transform categorical columns

In [140]:
preprocessor = DataPreprocessor(df)

In [141]:
preprocessor.one_hot_encode(columns=categorical_cols)

### Normalize numerical columns

In [142]:
preprocessor.normalization(columns=numerical_cols)

## Check and write data

In [143]:
cleaned_data = preprocessor.get_preprocessed_data()
cleaned_data

,Size,Frequency of Purchases,Age,Purchase Amount (USD),Review Rating,Previous Purchases,Gender_Male,Category_Clothing,Category_Footwear,Category_Outerwear,...,Shipping Type_Free Shipping,Shipping Type_Next Day Air,Shipping Type_Standard,Shipping Type_Store Pickup,Discount Applied_Yes,Preferred Payment Method_Cash,Preferred Payment Method_Credit Card,Preferred Payment Method_Debit Card,Preferred Payment Method_PayPal,Preferred Payment Method_Venmo
0,3,2,0.711538,0.4125,0.24,0.265306,1,1,0,0,...,0,0,0,0,1,0,0,0,0,1
1,3,2,0.019231,0.5500,0.24,0.020408,1,1,0,0,...,0,0,0,0,1,1,0,0,0,0
2,1,1,0.615385,0.6625,0.24,0.448980,1,1,0,0,...,1,0,0,0,1,0,1,0,0,0
3,2,1,0.057692,0.8750,0.40,0.979592,1,0,1,0,...,0,1,0,0,1,0,0,0,1,0
4,2,6,0.519231,0.3625,0.08,0.612245,1,1,0,0,...,1,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3895,3,1,0.423077,0.1000,0.68,0.632653,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
3896,3,3,0.653846,0.3625,0.80,0.816327,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3897,3,5,0.538462,0.1625,0.16,0.469388,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
3898,1,1,0.500000,0.7125,0.52,0.469388,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1


In [144]:
cleaned_data.to_csv('../data/cleaned_data.csv', index=False)